In [1]:
import numpy as np 
import cv2
from skimage.feature import local_binary_pattern

In [2]:
import os
directory_list = list()
dir_path = "dataset"
for root, dirs, files in os.walk(dir_path, topdown=False):
    for name in dirs:
        directory_list.append((name))
#print(directory_list)
for directory in directory_list:
    print(directory)

catagory_0
catagory_1
catagory_2
catagory_3
catagory_4
catagory_5
catagory_6
catagory_7


In [3]:
import glob

labels = 'labels.csv'
labelfile = open(labels,'w')
for i in range(len(directory_list)):
    readpath = (dir_path+'/' + directory_list[i]+'/*jpg')
    #print(readpath)
    images = glob.glob(readpath)
    for image in images:
        labelfile.write(image+','+directory_list[i][9:len(directory_list[i])]+'\n')
labelfile.close()

In [4]:
import random
labels = 'labels.csv'
shuffled_labels = 'Shuffled_labels.csv'

labelfile = open(labels, "r")
lines = labelfile.readlines()
labelfile.close()
random.shuffle(lines)

shufflefile = open(shuffled_labels, "w")
shufflefile.writelines(lines)
shufflefile.close()

In [5]:
#to check uniform or not 
def uniform(pattern):
    pat= int(pattern)
    a=0
    b=0
    cnt=0
    for i in range(0,8):
        if( i==0 ):
            a= int(pattern/2**(7-i))
        else:
            b= int(pattern/2**(7-i))
            if(b!=a):
                cnt=cnt+1
                a=b
        pattern=pattern%2**(7-i)
    if(cnt<=2):
        return 1 #uniform hbe
    else:
        return 0
items = []
true_items = []
for i in range(0, 256):
    if uniform(i):
        items.append(i)
        true_items.append(i)
        
items.append(250)
true_items.append(250)
print(len(items))


59


In [6]:
import cv2
import numpy as np
def get(image, idx, idy):
    if idx < (len(image)) and idy < len(image[0]) and idx>=0 and idy >=0:
        return 1
    else:
        return 0
    
def dir(x,y,img):
	ax=int(img[x+1,y])-int(img[x,y])
	ay=int(img[x,y-1])-int(img[x,y])
    
	if ax>=0 and ay>=0:
		return 1
	elif ax<0 and ay>=0:
		return 2
	elif ax<0 and ay<0:
		return 3
	elif ax>=0 and ay<0:
		return 4
		
def d_lbp(x,y,img):
    t = 40
    out=[]
    fx=[0,-1,-1,-1,0,1,1,1]
    fy=[1,1,0,-1,-1,-1,0,1]
    val = int(img[x][y])
    for i in range(0, 8):
        n_x = x+fx[i]
        n_y = y+fy[i]
        val1=int(img[n_x][n_y])
        val2 = int(img[n_x+fx[i]][n_y+fy[i]])
        if val2>= val1+t and val1>=val+t:
            out.append(4)
        elif val2<val1+t and val1>val+t:
            out.append(3)
        elif val2>=val1+t and val1<val+t:
            out.append(2)
        elif val2<val1+t and val1<val+t:
            out.append(1)
        else:
            out.append(0)
    return out

def add_dict(dic, val):
    if val not in dic.keys():
        dic[250] = dic[250] +1
    else:
        dic[val] = dic[val] + 1

def double_lbp(img, dic1, dic2, dic3, dic4):
    fx=[0,-1,-1,-1,0,1,1,1]
    fy=[1,1,0,-1,-1,-1,0,1]
    for x in range(2, len(img)-2):
        for y in range(2, len(img[x])-2):
            value = d_lbp(x, y, img)
            val1=0
            val2=0
            val3=0
            val4=0
            for i in range(len(value)):
                if(value[i]==1):
                    val1+=2**(7-i)
                elif(value[i]==2):
                    val2+=2**(7-i)
                elif(value[i]==3):
                    val3+=2**(7-i)
                elif(value[i]==4):
                    val4+=2**(7-i)
            add_dict(dic1, val1)
            add_dict(dic2, val2)
            add_dict(dic3, val3)
            add_dict(dic4, val4)
    return dic1, dic2, dic3, dic4

In [7]:
import cv2
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from scipy.stats import itemfreq
import time
from functools import wraps

df = pd.read_csv(shuffled_labels, header=None)
rows = df.iterrows()

X_addrs = []
X_hist = []
Y_hist = []
#row = rows[0]
j = 0


# uncomment below this for local tetra pattern
dic1 = {}
dic2 = {}
dic3 = {}
dic4 = {}

def create_dic():
    for i in range(len(items)):
        dic1[items[i]] = 0
        dic2[items[i]] = 0
        dic3[items[i]] = 0
        dic4[items[i]] = 0

start_time = time.time()      
for row in rows:
    #print(row[1][1])
    create_dic()
    img = cv2.imread(row[1][0])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    new_x = []
    dic1, dic2, dic3, dic4 = double_lbp(img,dic1, dic2, dic3, dic4)
    
    x1 = []
    for i in dic1.keys():
        x1.append(dic1[i])
    x1 = np.array(x1)
    x1 = x1/np.sum(x1)
    
    x2 = []
    for i in dic2.keys():
        x2.append(dic2[i])
    x2 = np.array(x2)
    x2 = x2/np.sum(x2)
    
    x3 = []
    for i in dic3.keys():
        x3.append(dic3[i])
    x3 = np.array(x3)
    x3 = x3/np.sum(x3)
    
    x4 = []
    for i in dic4.keys():
        x4.append(dic4[i])
    x4 = np.array(x4)
    x4 = x4/np.sum(x4)
    
    new_x = x1.tolist()+x2.tolist()+x3.tolist()+x4.tolist()
    #new_x = x1.tolist()+x2.tolist()+x4.tolist()
    
    X_hist.append(new_x)
    X_addrs.append(row[1][0])
    Y_hist.append(row[1][1])
    if j%50==0:
        elapsed_time = time.time() - start_time
        print(str(j)+" done in " + str(elapsed_time))
        start_time = time.time()
    j = j + 1

Using TensorFlow backend.


0 done in 1.3670732975006104
50 done in 35.34597992897034
100 done in 30.923363208770752
150 done in 28.747615814208984
200 done in 39.798649311065674
250 done in 40.85354518890381
300 done in 32.067694664001465
350 done in 36.91008114814758
400 done in 25.774287700653076
450 done in 26.377923727035522
500 done in 25.740319967269897
550 done in 26.23100972175598
600 done in 26.293989181518555
650 done in 27.221457719802856
700 done in 26.842676401138306
750 done in 25.572420358657837
800 done in 26.60779309272766
850 done in 26.219033002853394
900 done in 27.338391542434692
950 done in 34.214465379714966
1000 done in 26.912655353546143
1050 done in 25.785279989242554
1100 done in 26.296988248825073
1150 done in 25.82124423980713
1200 done in 27.047574520111084
1250 done in 27.009581089019775
1300 done in 25.58339548110962
1350 done in 26.175041675567627
1400 done in 25.67735767364502
1450 done in 27.86709213256836
1500 done in 26.627801418304443


In [8]:
print(X_addrs[0])
print(X_hist[0])
print(Y_hist[0])
#print(X_hist)
X_hist = np.array(X_hist)
Y_hist = np.array(Y_hist)
print(len(X_addrs))
print(len(X_hist[50]))
print(X_hist.shape)
#print((Y_hist))

dataset/catagory_0\D112_0_22.jpg
[0.0012356919875130073, 0.0003251821019771072, 0.0005202913631633715, 0.0003902185223725286, 0.0008454734651404787, 0.0003902185223725286, 0.0001951092611862643, 0.0007804370447450572, 0.0006503642039542144, 0.0009755463059313215, 0.0007804370447450572, 6.503642039542144e-05, 0.0005202913631633715, 0.0003251821019771072, 0.00045525494276795007, 0.0009755463059313215, 0.0007154006243496358, 0.0005202913631633715, 0.0009755463059313215, 0.0020161290322580645, 0.007479188345473465, 0.006698751300728408, 0.0001951092611862643, 0.0006503642039542144, 0.0008454734651404787, 0.0013007284079084287, 0.0028616024973985433, 0.009755463059313215, 0.005072840790842872, 0.0012356919875130073, 0.0001951092611862643, 0.0007154006243496358, 0.0012356919875130073, 0.006243496357960458, 0.007088969823100937, 0.023868366285119666, 0.0003902185223725286, 0.00026014568158168577, 0.00045525494276795007, 0.0015608740894901144, 0.006113423517169615, 0.004617585848074922, 0.0006

In [9]:
X_hist = np.array(X_hist)
Y_hist = np.array(Y_hist)

print(X_hist.shape, Y_hist.shape)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(np.array(X_hist), np.array(Y_hist), test_size=0.3)
X_train = np.array(X_train)
print(len(X_train))

'''
for i in range(len(X_train)):
    if(len(X_train[i]) != 26):
       print(len(X_train[i]))
'''

print(len(Y_train))

(1504, 236) (1504,)
1052
1052


In [21]:
#this code is to use the distance function
def feature_distance(feature1, feature2):
    dist = 0.0
    for i in range(len(feature1)):
        dist += abs((feature1[i]*1.0-feature2[i]*1.0)/(1.0+feature1[i]*1.0+feature2[i]*1.0))
        #print(dist)
    return dist
x = X_hist
print(x.shape)
#print(unique_name[X_addrs[0]])
#print(Y_hist[0])
#print(Y_hist[1])
#print(feature_distance(x[0], x[10]))

#60 -> 0.
#50->0.
#40->0.
#35->0.
#30->0.


def clc():
    query_length = 40
    sm = 0
    run = 20
    for j in range(run):
        true_val = 0
        false_val = 0
        distance_list = []
        query = x[j]
        query_index = j
        for i in range(len(Y_hist)):
            distance_list.append(feature_distance(query, x[i]))
        unsorted = zip(distance_list, Y_hist)
        sorted_touple = sorted(unsorted, key = lambda element : element[0])
        #print(len(sorted_touple))
        for i in range(query_length):
            if(sorted_touple[i][1] == Y_hist[query_index]):
                true_val = true_val + 1
            else:
                #print("this is wrong "+str(i)+" confusing with "+str(sorted_touple[i][1]))
                false_val = false_val+1
        sm = sm + true_val*1.0/query_length*1.0
        print(true_val*1.0/query_length*1.0)
    return sm/run
print("avg = ", clc())

(1504, 236)
1.0
0.25
1.0
0.95
1.0
1.0
1.0
1.0
1.0
1.0
0.75
1.0
0.825
0.725
1.0
0.775
0.9
1.0
0.525
1.0
avg =  0.8849999999999998


In [146]:
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
#clf = svm.SVC()
#clf = GaussianNB()
#clf = LogisticRegression()
#clf = MLPClassifier(alpha=1)
#clf = AdaBoostClassifier()
clf = RandomForestClassifier()
#clf = DecisionTreeClassifier()

#X_train = np.array(X_train)
print(type(X_train))

clf.fit(X_train,Y_train)

clf.score(X_test, Y_test)
#0.96276595744680848
#with threshhold 50 and 4*59 feature

<class 'numpy.ndarray'>


0.85119518758904544

In [147]:
import pickle
from sklearn.model_selection import cross_val_score
#clf = pickle.load(open('RandomForest_model.sav', 'rb'))
clf = RandomForestClassifier()
clf.fit(X_train,Y_train)
scores = cross_val_score(clf,X_test,Y_test,cv=5)
print((scores))
clf.score(X_test, Y_test)

[ 0.74618902  0.78010878  0.75596184  0.77849636  0.76778414]


0.85356973246794365

In [148]:
import pickle
#for saving model
filename = 'my_double_lbp_model.sav'
pickle.dump(clf, open(filename, 'wb'))
clf = pickle.load(open('my_double_lbp_model.sav', 'rb'))
clf.score(X_test, Y_test)
#scores = cross_val_score(clf, X_test, Y_test, cv=20)
#print((scores))

0.85356973246794365

In [149]:
import numpy as np

X = np.array([[-1, -1], [-2, -1], [1, 1], [2, 1]])
y = np.array([1, 1, 2, 2])

from sklearn.svm import SVC

clf = SVC()
print(type(y))
clf.fit(X, y) 
print(clf.predict([[-0.8, -1]]))

<class 'numpy.ndarray'>
[1]


In [150]:
a = [0]*26
a = np.array(a)
b = a.copy()
b = [0]*14
b = np.array(b)
print(b.shape)
c = a+b
print(c)

(14,)


ValueError: operands could not be broadcast together with shapes (26,) (14,) 

In [ ]:
#lbp-> Local derivative ternary lbp last e  tetra 